# 文件IO操作

## mode 

### 主模式
- r  readonly
  r模式打开，**指针**在0
  - 文件不存在，FileNotFoundError
  - 文件存在，以只读方式打开，如果使用write会报错
   
- w  不可读，可写
  w模式打开，**指针**永远是0也是EOF
  - 文件存在，则清空文件
  - 文件不存在，创建一个新文件，可以写
  - **w模式 实际上提供的就是一张白纸**
  
- x  Exist 不支持读取，可写
  x模式打开，**指针**永远是0也是EOF
  - 打开的时候，如果文件存在，抛出FileExistsError
  - 如果文件不存在，创建全新文件
  
- a  append 可以写，不可读
  EOF end of file append，a模式打开，**指针**指向EOF
  - 如果文件存在，追加写入
  - 如果文件不存在，创建全新文件，追加写入
  
### 附加模式
  - t text 文件，字符流
    - rt：缺省模式，省略为r，只读文本打开(readonly text)
    - wt: 省略为w，只写打开
    - xt
    - at
  - b binary 二进制，字节流
    - rb
    - wb
    - xb
    - ab
  - + 依赖模式，和主模式配合，不影响使用t和b，影响主模式读写，给主模式补充一种缺失的能力
    - r+：增加了写能力
    - w+: 增加了读取能力，但是还是把存在得文件清空了
    - x+
    - a+
 
## 文件缓冲区 问题
  - 文件没有关闭**close**或者**flush**，输出的内容先进入了写缓冲，所以，我们不一定能够立即看到内容
  - 需要查看，关闭 或者 调用flush方法即可
  - close时，会自动调用flush，将缓冲区buffer数据直接写入磁盘
  
  
## 文件指针
  * tell seek操作的都是字节偏移
    - tell 告诉你字节偏移
    - seek 跳到指定字节处，一定不能左超界，可以右超界。whence为0不能用负数
  
  * 文本模式下， whence取1或者2的时候，只能使用0
      - f.seek(0, 2) 指针回到EOF
      - f.seek(0, 1) 指针原地不动
      - f.seek(100)  正向可以超界
  * 二进制模式下，whence取1和2时，可正可负
      - f.seek(0, 2)     EOF
      - f.seek(100, 2)   EOF + 100
      - f.seek(-100, 2)  EOF - 100 注意，不能左超界
      - f.seek(0, 1)     原地不动
      - f.seek(100, 1)   cur + 100
      - f.seek(-100, 1)  cur - 100 注意，不能左超界
  
  * 常用：
    - f.seek(0)    回到开头，重新读取数据
    - f.seek(0, 2) 回到EOF，准备追加数据
    - f.seek(100)


## read
  - read(size=-1)
    - size表示读取的多少个**字符(text模式)或者字节(binary模式)**；负数或者None表示读取到EOF


[字符集、字符编码详解](https://www.cnblogs.com/niceforbear/articles/4534784.html)

```
r    只读不能写，默认模式
awx  只写，它们都可以创建新文件
rwxa 互斥：ValueError: must have exactly one of create/read/write/append mode
    
主模式，还是比较强势，听它的

不能单独使用t or b，不许使用一个主模式与其配合
不管哪一种主模式，t被使用，操作的都是字符串，字符串和编码有关。中文占几个字节，和当前打开文件时的编码有关。win:cp936|gbk linux:UTF-8
        
写文本文件，应该保证写入的编码统一，encoding='utf-8'。建议大家，文本模式指定编码，也使用utf-8


'啊'.encode() => b'\xb0\xa1'
t: 文本模式操作的都是字符 读按字符读 写按字符写   
    read(1) => '啊'
    
b: 二进制模式操作都是字节byte 
    read(1) => '\xb0'
```

In [134]:
pwd

'D:\\Docs\\TrainPath\\PythonNote'

In [137]:
%ls

 驱动器 D 中的卷是 数据
 卷的序列号是 3CF4-331D

 D:\Docs\TrainPath\PythonNote 的目录

2020/12/08  18:30    <DIR>          .
2020/12/08  18:30    <DIR>          ..
2020/12/08  18:16    <DIR>          .ipynb_checkpoints
2020/11/16  13:55             1,691 PYTHON.md
2020/12/08  18:30                 0 test
2020/12/08  18:13             6,443 测试.ipynb
2020/12/08  18:13           392,240 基础_函数装饰器.ipynb
2020/12/08  18:30             1,571 文件IO.ipynb
2020/11/16  09:57            29,249 直播笔记_20201114.ipynb
               6 个文件        431,194 字节
               3 个目录 772,952,739,840 可用字节


In [143]:
!echo 134 > test 

In [144]:
!cat test

134 


In [146]:
f = open('test')    # open方法打开成功后，会得到一个文件对象

In [147]:
f    
# windows 默认编码 cp936, code page 等价认为 GBK
# linux   默认编码 UTF-8

<_io.TextIOWrapper name='test' mode='r' encoding='cp936'>

In [148]:
f.close()    # 每一个文件打开 会占用文件描述符 不用之后切记关闭

In [149]:
f = open('test')

In [151]:
f.read()

'134 \n'

In [152]:
f.write('abc')    # 不支持写入

UnsupportedOperation: not writable

In [156]:
f.close()

In [157]:
!cat test

134 


In [158]:
f.closed    # 判断文件是否被关闭

True

In [168]:
# w
f = open('test', 'w')

In [163]:
f.write('xyzb')    # 返回写入的长度

4

In [162]:
f.read()

UnsupportedOperation: not readable

In [170]:
f.close()

In [171]:
!cat test

In [172]:
f1 = open('test1', 'w')

In [175]:
f1.closed

True

In [174]:
f1.close()

In [177]:
f2 = open('test1', 'x')

FileExistsError: [Errno 17] File exists: 'test1'

In [180]:
f2 = open('test2', 'x')

In [182]:
f2.write('abc')    # 写入缓冲区问题

3

In [181]:
f2.read()

UnsupportedOperation: not readable

In [185]:
f2.close()
# 没有关闭或者flush，输出的内容先进入了缓冲区
# close() 方法会调用一次flush

In [186]:
f3 = open('test2', 'a')

In [187]:
f3.write('123')

3

In [188]:
f3.read(0)

UnsupportedOperation: not readable

In [189]:
!cat test2

abc


In [190]:
f3.flush()

In [191]:
!cat test2

abc123


In [193]:
f3.close()

In [194]:
f4 = open('test3', 'a')

In [198]:
f4.readable(), f4.writable()

(False, True)

In [199]:
f4.close()

In [200]:
f5 = open('test', 'rw')    # rwxa 互斥

ValueError: must have exactly one of create/read/write/append mode

In [201]:
!cat test2

abc123


In [202]:
f2 = open('test2', 'r')    # readonly

In [203]:
f2.read()    # => str

'abc123'

In [204]:
f2.close()

In [205]:
f2 = open('test2', 'rt')  # readonly text

In [206]:
f2

<_io.TextIOWrapper name='test2' mode='rt' encoding='cp936'>

In [207]:
f2.read()    # => str

'abc123'

In [ ]:
字符串的世界 跟编码有关

In [209]:
f2.close()

In [210]:
f3 = open('test2', 'wt')    # 只写文本打开

In [211]:
!cat test2  # 内容被清空，w模式的特点

In [212]:
f3.write('abc')

3

In [213]:
f3.write(b'123')    # 不能是 bytes

TypeError: write() argument must be str, not bytes

In [214]:
f3.close()

In [215]:
f3 = open('test2', 'wt')    # 只写文本打开

In [216]:
f3

<_io.TextIOWrapper name='test2' mode='wt' encoding='cp936'>

In [218]:
f3.write('xyz111')

6

In [222]:
!cat test2

xyz111中


In [219]:
f3.write('中')    # return: 1 一个字符，对应2个字节

1

In [223]:
f3.close()

In [224]:
!cat test2

xyz111中


In [225]:
f4 = open('test2', 'ab')

In [226]:
f4

<_io.BufferedWriter name='test2'>

In [227]:
f4.write('啊')

TypeError: a bytes-like object is required, not 'str'

In [228]:
f4.write('啊'.encode())

3

In [232]:
'啊'.encode()    # encode() 缺省：uft-8

b'\xe5\x95\x8a'

In [233]:
'啊'.encode(encoding='gbk') 

b'\xb0\xa1'

In [234]:
f4.close()

In [235]:
!cat test2

xyz111中鍟�


In [239]:
f5 = open('test2', 'rb')

In [240]:
f5.read()

b'xyz111\xd6\xd0\xe5\x95\x8a'

In [238]:
f5.close()

In [242]:
'寰堝睂'.encode('gbk')

b'\xe5\xbe\x88\xe5\xb1\x8c'

In [250]:
b'\xe5\xbe\x88\xe5\xb1\x8c'.decode('utf-8')

'很屌'

In [257]:
'中'.encode('utf8')

b'\xe4\xb8\xad'

In [259]:
f5.close()

In [ ]:
+ 附加模式

In [260]:
f = open('test', 'w+')

In [ ]:
f.write('0123456789')

In [272]:
!cat test

abc3456789


In [264]:
f.close()

In [273]:
f2 = open('test', 'r+')
f2.write('xxx')

3

In [274]:
f2.close()

In [280]:
!cat test

xxx3456789end


In [276]:
f3 = open('test', 'a+')  # IO 占文件描述符fd，用完记得关闭
print(f3.read())    # 读不到内容 文件指针在EOF

In [278]:
f3.write('end')

3

In [281]:
f3.close()

In [288]:
f3 = open('test', 'a+')

In [291]:
f3.tell()    # tell & seek 不管你使用的是b还是t，它永远说的是字节的偏移

2

In [287]:
!cat test

中


In [295]:
f3.seek(0, 0)    # 相对于开始 whence 0：开始 1：当前位置 2：EOF

0

In [293]:
f3.seek(cookie, whence=0, /)

* 0 -- start of stream (the default); offset should be zero or positive
* 1 -- current stream position; offset may be negative
* 2 -- end of stream; offset is usually negative

In [296]:
f3.read()    # read 从当前指针位置向后读取

'中'

In [328]:
f3.seek(0, 2) 

2

In [331]:
f3.read()

'中'

In [320]:
f3.tell()

2

In [330]:
f3.seek(0)

0

In [338]:
f3.seek(-1, 0)

ValueError: negative seek position -1

In [340]:
f3.close()

In [418]:
f = open('test', 'rb+') #, encoding='utf-8')

In [351]:
f.read()

'abcxyz\n'

In [352]:
f.write('xyz')

3

In [355]:
!cat test

abcxyz
xyz


In [393]:
f.read()  # read, seek 的时候 会调用一次flush

b'abcxyz\r\nxyz\xb0\xa1\xb0\xa1'

In [360]:
hex(ord('啊')), '啊'.encode(), '啊'.encode('gbk')

('0x554a', b'\xe5\x95\x8a', b'\xb0\xa1')

In [361]:
ord?

In [401]:
f.write(b'\xe5\x95\x8a')

3

In [422]:
f.close()

In [410]:
f.read(4096)  # 100G打文件 4k 4k循环读

In [412]:
f.writelines?

In [413]:
# write()
'\n'.join(['abc', '123', 'magedu'])

'abc\n123\nmagedu'

In [414]:
# writelines()
[x + '\n' for x in ['abc', '123', 'magedu']]

['abc\n', '123\n', 'magedu\n']

In [423]:
f.fileno()

ValueError: I/O operation on closed file

In [424]:
l = [0, 3, 7, 17, 1, 6, 19, 7, 13, 9, 20, 17, 12, 10, 20, 16, 11, 12, 9, 6]

In [431]:
l.count(7)

2

In [ ]:
* 上下文管理

In [435]:
with open('test') as f:
    print(f)
    print(f.closed)
    print(f.read())
    print(f.write('abc'))

with 文件对象 as 标识符：
    语句块

<_io.TextIOWrapper name='test' mode='r' encoding='cp936'>
False
abcxyz
xyz啊啊


UnsupportedOperation: not writable

In [436]:
f.closed

True

In [ ]:
with 语法，某些对象的上下文配合，这个对象要支持上下文
上下文在Python种应用非常广泛，必须掌握

with 文件对象 as 标识符：
    语句块

In [440]:
f1 = open('test')
with f1:
    # print(f1 is f2)
    print(f1.closed)
    # print(f2.closed)

False


In [441]:
f1.closed, f2.closed

(True, True)

In [ ]:
with 文件对象    
with 后面写支持上下文管理的文件对象, as子句为这个对象起别名
当 with 的语句块中所有的语句执行完毕后，它会把with后这个文件对象调用它的close
方法，而且不管 with 语句块中是否出现异常，都会保证调用 

上下文管理的语句 并不会开启新的作用域

# 下面的等价式 仅仅适用于文件对象
# 1.
with open('test') as f  => f = with后的文件对象 => f = open('test')

# 2.
f1 = open('test')
with f1 as f2

In [456]:
with open('test') as f:
    for line in f:
        print(type(line), line.rstrip())

<class 'str'> abcxyz
<class 'str'> xyz啊啊
<class 'str'> Kathr1ne
<class 'str'> Minho


In [ ]:
text   模式 逐行读取
binary 模式 按字节读取 f.read(4096)

In [462]:
next(map(str, range(100, 120)))

'100'

In [467]:
dict(map(lambda x: (x, x*2), range(100, 105)))

{100: 200, 101: 202, 102: 204, 103: 206, 104: 208}

In [ ]:
* 路径操作

In [468]:
from os import path    
# from os import path => path.xxx   
# import os           => os.path.xxx
# import os.path      => os.path.xxx
# import 后面必须是模块，不可以是类或者函数 或者变量等

In [481]:
p1 = path.join('/etc', 'a', 'b')
# 路径分隔符
# windows \，但是它也认 /
# linux|*nix /

In [482]:
type(p1), p1

(str, '/etc\\a\\b')

In [483]:
path.exists(p1)

False

In [484]:
path.split(p1)    # tuple （父目录，基名）

('/etc\\a', 'b')

In [486]:
path.dirname(p1), path.basename(p1)

('/etc\\a', 'b')

In [488]:
p2 = path.join(path.dirname(p1), 'mysql.tar.gz')

In [489]:
path.splitdrive('c:/windows/nt/etc')

('c:', '/windows/nt/etc')

In [491]:
path.splitext(p2)[0] + '.xz'  # 获取文件扩展名

'/etc\\a\\mysql.tar.xz'

In [493]:
p3 = '/etc/a/b/c'

In [494]:
path.dirname(p3)

'/etc/a/b'

In [495]:
path.dirname('/etc/a/b')

'/etc/a'

In [498]:
path.dirname('/etc/a'), path.dirname('/etc'), path.dirname('/')

('/etc', '/', '/')

In [512]:
p3 = '/etc/a/b/c'
print(p3)
while p3 != path.dirname(p3):
    p3 = path.dirname(p3)
    print(p3)

/etc/a/b/c
/etc/a/b
/etc/a
/etc
/


In [516]:
path.dirname('/etc/a/b/c')

'/etc/a/b'

In [518]:
path.dirname('/etc/a/b'), path.dirname('/etc')

('/etc/a', '/')

In [519]:
p4 = 'a/b/c'    # 相对路径 相对于当前工作路径

In [521]:
path.abspath(p4)

'D:\\Docs\\TrainPath\\PythonNote\\a\\b\\c'

In [524]:
pwd    # jupyter magic %pwd

'D:\\Docs\\TrainPath\\PythonNote'

In [525]:
p3 = 'etc/a/b/c'
print(p3)
while p3 != path.dirname(p3):
    p3 = path.dirname(p3)
    print(p3)

etc/a/b/c
etc/a/b
etc/a
etc



In [527]:
from pathlib import Path    
# Path 类，全面向对象，用起来就是用.xxx就可以
# 底层依赖：os模块 os.path模块
# 一切都使用Path对象
# 根据操作系统的不同，返回不同路径类型的对象
# 它的操作一般返回的还是路径对象

In [530]:
p1 = Path()

In [531]:
p1

WindowsPath('.')

In [532]:
p1.absolute()

WindowsPath('D:/Docs/TrainPath/PythonNote')

In [533]:
Path(), Path('.'), Path('')    # 都代表当前目录

(WindowsPath('.'), WindowsPath('.'), WindowsPath('.'))

In [538]:
p2 = Path('a', 'b/c', 'd', Path('e/f', 'g'))

In [535]:
Path('a', 'b/c', 'd', Path('e/f', 'g'), '/')  # 直接变成根目录 /

WindowsPath('/')

In [536]:
Path('a', 'b/c', 'd', Path('e/f', '/g'))

WindowsPath('/g')

In [537]:
Path('/a', 'b/c', 'd', Path('e/f', 'g'))

WindowsPath('/a/b/c/d/e/f/g')

In [539]:
p2

WindowsPath('a/b/c/d/e/f/g')

In [540]:
Path('a', 'b/c') / 'd' / 'e/f'    # 运算符重载 这里用作路径拼接

WindowsPath('a/b/c/d/e/f')

In [541]:
Path('a', 'b/c/d') / 'e/g'

WindowsPath('a/b/c/d/e/g')

In [542]:
Path('a') / Path('b/c')

WindowsPath('a/b/c')

In [543]:
'a' / Path('b/c')

WindowsPath('a/b/c')

In [545]:
'a/b' / Path('/') / 'c' / 'd/f'

WindowsPath('/c/d/f')

In [546]:
'a' / Path('b/c') / Path('d') / 'e'

WindowsPath('a/b/c/d/e')

In [549]:
'a/b' / Path('c/d', 'e', Path('f/g', 'h'))

WindowsPath('a/b/c/d/e/f/g/h')

In [550]:
'a/b' / 'c' / Path('d', 'e/f')

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [554]:
'a/b' / Path('c/d', 'f', Path('g/h')) / 'i' / Path('j/k', 'l')

WindowsPath('a/b/c/d/f/g/h/i/j/k/l')

In [ ]:
Path / str    # 运算符重载 内部处理 返回 Path 对象

Path / str  => Path obj
Path / Path => Path obj
str / Path  => Path obj    内部反转，等价：Path / str
str / str   => TypeError

In [558]:
'a/b' / ('c' / Path('d', 'e/f')) 

WindowsPath('a/b/c/d/e/f')

In [559]:
'a/b' / ('c' / Path('d', 'e/f')) + 'g' # + 没有做运算符重载

TypeError: unsupported operand type(s) for +: 'WindowsPath' and 'str'

In [561]:
p3 = Path('a/b/c/d/e/f') # WindowsPath | PosixPath

In [565]:
p3.parts

('a', 'b', 'c', 'd', 'e', 'f')

In [567]:
Path('/etc/sysconfig/network/ifcfg-eth0').parts

('\\', 'etc', 'sysconfig', 'network', 'ifcfg-eth0')

In [568]:
Path('/etc/sysconfig/network/ifcfg-eth0').parts[-1]  # basename

'ifcfg-eth0'

In [571]:
(Path('/data/projects') / 'webapps').joinpath('ROOT', Path('index.html'))

WindowsPath('/data/projects/webapps/ROOT/index.html')

In [576]:
p5 = (Path('/data/projects') / 'webapps').joinpath('ROOT', 'index.html')

In [577]:
p5

WindowsPath('/data/projects/webapps/ROOT/index.html')

In [578]:
p5.parts

('\\', 'data', 'projects', 'webapps', 'ROOT', 'index.html')

In [579]:
p5.name    # os.path.basename => str

'index.html'

In [582]:
p5.parent  # os.path.dirname => str

WindowsPath('/data/projects/webapps/ROOT')

In [585]:
p5.name, p5.stem, p5.suffix    # name = stem + suffix

('index.html', 'index', '.html')

In [593]:
Path('a/b/c/d/').name, Path('a/b/c/d/').parent

('d', WindowsPath('a/b/c'))

In [592]:
Path().name, Path('/').name

('', '')

In [596]:
Path().parent, Path('/').parent

(WindowsPath('.'), WindowsPath('/'))

In [597]:
p5

WindowsPath('/data/projects/webapps/ROOT/index.html')

In [601]:
p6 = p5.with_name('mysql.tar.gz')

In [602]:
p6

WindowsPath('/data/projects/webapps/ROOT/mysql.tar.gz')

In [606]:
p5.parent / 'mysql.tar.gz'

WindowsPath('/data/projects/webapps/ROOT/mysql.tar.gz')

In [603]:
p7 = p5.with_suffix('.jsp')

In [604]:
p7

WindowsPath('/data/projects/webapps/ROOT/index.jsp')

In [609]:
p6.suffixes

['.tar', '.gz']

In [612]:
p1 = Path('/etc/myshell/p2p_cfg') / 'p2p_pkg.sh'

In [620]:
p1.parent, p1.name, p1.suffix, p1.stem

(WindowsPath('/etc/myshell/p2p_cfg'), 'p2p_pkg.sh', '.sh', 'p2p_pkg')

In [624]:
p1.absolute()

WindowsPath('D:/etc/myshell/p2p_cfg/p2p_pkg.sh')

In [626]:
p1.with_name('new_natfullcone.sh')

WindowsPath('/etc/myshell/p2p_cfg/new_natfullcone.sh')

In [628]:
p1.with_suffix('.tar.xz')

WindowsPath('/etc/myshell/p2p_cfg/p2p_pkg.tar.xz')

In [631]:
list(p1.parents)    # 不是迭代器

[WindowsPath('/etc/myshell/p2p_cfg'),
 WindowsPath('/etc/myshell'),
 WindowsPath('/etc'),
 WindowsPath('/')]

In [632]:
list(Path('a/b/c/d/e').parents)  # 父路径迭代方式 由近及远

[WindowsPath('a/b/c/d'),
 WindowsPath('a/b/c'),
 WindowsPath('a/b'),
 WindowsPath('a'),
 WindowsPath('.')]

In [642]:
# 不可以超界
# 不支持负索引  if idx < 0 or idx >= len(self)
Path('a/b/c/d/e').parents[0]

WindowsPath('a/b/c/d')

In [643]:
Path('a/b/c/d/e').parent

WindowsPath('a/b/c/d')

In [644]:
list(Path('a/b/c/d/e').absolute().parents)  # 路径对象

[WindowsPath('D:/Docs/TrainPath/PythonNote/a/b/c/d'),
 WindowsPath('D:/Docs/TrainPath/PythonNote/a/b/c'),
 WindowsPath('D:/Docs/TrainPath/PythonNote/a/b'),
 WindowsPath('D:/Docs/TrainPath/PythonNote/a'),
 WindowsPath('D:/Docs/TrainPath/PythonNote'),
 WindowsPath('D:/Docs/TrainPath'),
 WindowsPath('D:/Docs'),
 WindowsPath('D:/')]

In [ ]:
# 家目录 当前路径 不会因为你当前路径是谁而改变 相当于常量

In [647]:
p1.cwd(), p2.cwd()

(WindowsPath('D:/Docs/TrainPath/PythonNote'),
 WindowsPath('D:/Docs/TrainPath/PythonNote'))

In [648]:
p1.home(), p2.home()

(WindowsPath('C:/Users/97431'), WindowsPath('C:/Users/97431'))

In [650]:
Path.cwd(), Path.home()

(WindowsPath('D:/Docs/TrainPath/PythonNote'), WindowsPath('C:/Users/97431'))

In [654]:
import os.path

In [ ]:
* 判断方法

In [655]:
p1.home().exists(), os.path.exists('C:/Users/97431')

(True, True)

* 绝对路径
    * resolve()   非windows 返回一个新的路径，这个新路径就是当前Path对象的绝对路径，如果软连接则直接被解析
    * absolute()  获取绝对路径
    
* 判断
    p1.exists()
    p1.is_absolute()
    p1.is_dir()
    p1.is_block_device()
    p1.is_char_device()
    p1.is_symlink()
    p1.is_socket()
    
stat  相当于stat命令
lstat 使用方法同stat() 但如果是符号链接，则显示符号链接本身的文件信息

In [ ]:
* 通配符
Path().glob('**/*.py')
Path().rglob('*.py')  # r 递归

In [ ]:
Path('a/b/c/d/e').parent.mkdir(parents=True, exist_ok=True)
# parnets=True => mkdir -p

In [ ]:
Path().iterdir()    # 遍历

In [688]:
for i in [1, 2, 3, 4, 5]:
    print(i)
    break
else:
    print('22222')

1


In [690]:
not False

True

In [691]:
!cat test

abcxyz
xyz啊啊
Kathr1ne     
Minho


In [695]:
f = open('test', 'rb')
f.read(2)

b'ab'

In [696]:
f.tell()    # 文件指针 打开之后 每次读到那儿 问价指针停在那儿

2

In [714]:
if not f.read(10):
    print("NULL")

NULL


In [713]:
f.tell()

37

In [716]:
f.close()

In [717]:
import os.path

In [719]:
os.path.dirname('/etc/myshell/a.txt'), os.path.basename('/etc/myshell/a.txt')

('/etc/myshell', 'a.txt')

In [ ]:
* shutil模块


shutil.copy()    # copyfile() && copymode()
shutil.copy2()   # copyfile() && copystat()

In [720]:
import os

In [728]:
names = os.listdir('D:/Docs')

In [735]:
set(filter(lambda name: name.endswith('.md'), names))

{'Xbox修复.md', 'rpmbuild.md', 'sk.md', '案列.md', '海外项目相关.md'}

In [737]:
{name for name in names if name.endswith('.md')}

{'Xbox修复.md', 'rpmbuild.md', 'sk.md', '案列.md', '海外项目相关.md'}

In [ ]:
import shutil
# copytree
def ignore(src, names):
    # 忽略.txt结尾的
    ig = filter(lamdba x: x.endswith('.txt'), names)  
    return set(ig)    
# ig是迭代器 只能迭代一次 copytree函数如果2次以上迭代
# 不包装为集合会出问题

shutil.copytree('o:/temp', 'o:/tt/o', ignore=ignore)